In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
from sklearn.metrics import classification_report

2024-03-22 15:54:08.822870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:54:08.822892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:54:08.823375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/OpenWaste/Experimental-big-9/'
epochs = 50
batch_size = 16
lr = 0.01
weight_decay = None

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=seed),
    tf.keras.layers.RandomRotation(factor=0.05, seed=seed),
    tf.keras.layers.RandomContrast(factor=0.2, seed=seed)
])

In [5]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(dataset_path + '/train', batch_size=batch_size)
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(64, seed=seed).prefetch(tf.data.AUTOTUNE)

Found 370 files belonging to 9 classes.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 153 files belonging to 9 classes.


In [7]:
class_names = train_ds_raw.class_names
num_classes = len(class_names)

In [8]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(
    weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.momentum = 0.90

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, weight_decay=weight_decay)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/efficientnet_v2_open_waste.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
    
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/50


2024-03-22 15:54:16.838664: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1711137259.939608  227052 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 16s 130ms/step - loss: 1.8804 - accuracy: 0.3703 - val_loss: 1.1877 - val_accuracy: 0.7647
Epoch 2/50
24/24 [==============================] - 3s 73ms/step - loss: 0.8562 - accuracy: 0.7784 - val_loss: 0.6693 - val_accuracy: 0.8235
Epoch 3/50
24/24 [==============================] - 3s 75ms/step - loss: 0.3914 - accuracy: 0.9216 - val_loss: 0.4713 - val_accuracy: 0.8758
Epoch 4/50
24/24 [==============================] - 3s 77ms/step - loss: 0.2010 - accuracy: 0.9459 - val_loss: 0.3998 - val_accuracy: 0.8954
Epoch 5/50
24/24 [==============================] - 3s 57ms/step - loss: 0.1477 - accuracy: 0.9676 - val_loss: 0.3755 - val_accuracy: 0.8954
Epoch 6/50
24/24 [==============================] - 3s 56ms/step - loss: 0.1119 - accuracy: 0.9703 - val_loss: 0.3434 - val_accuracy: 0.8824
Epoch 7/50
24/24 [==============================] - 3s 57ms/step - loss: 0.0804 - accuracy: 0.9811 - val_loss: 0.3624 - val_accuracy: 0.8758
Epoch 8/50
24/24 [====

In [10]:
saved_model_path = 'models/efficientnet_v2_open_waste.keras' #model_checkpoint_callback.filepath
saved_model = tf.keras.models.load_model(saved_model_path)
saved_model

In [11]:
saved_model.evaluate(val_ds)

10/10 [==============================] - 2s 22ms/step - loss: 0.3068 - accuracy: 0.9020


[0.3068206012248993, 0.9019607901573181]

In [12]:
test_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 153 files belonging to 9 classes.


In [13]:
saved_model.evaluate(test_ds)

10/10 [==============================] - 1s 11ms/step - loss: 0.3068 - accuracy: 0.9020


[0.3068210482597351, 0.9019607901573181]

In [14]:
labels = []
predictions = []

for X, y in test_ds:
    y_pred = tf.argmax(saved_model.predict(X, verbose=0), axis=1).numpy().tolist()
    labels.extend(y.numpy().tolist())
    predictions.extend(y_pred)

In [15]:
print(classification_report(labels, predictions, target_names=test_ds.class_names))

                                    precision    recall  f1-score   support

          Cardboard CORRUGATED box       0.83      0.33      0.48        15
               Metal TIN container       0.80      1.00      0.89        12
Paper COATED PAPERBOARD coffee cup       0.82      1.00      0.90        14
    Plastic ALUMINUM LINED wrapper       0.80      0.89      0.84        18
     Plastic TYPE 1 food container       0.94      0.97      0.95        30
                Plastic TYPE 4 bag       0.93      1.00      0.97        14
    Plastic TYPE 5 sauce container       1.00      0.93      0.96        14
         Plastic TYPE 6 coffee lid       0.94      0.94      0.94        18
 Plastic TYPE 6 straws and cutlery       1.00      1.00      1.00        18

                          accuracy                           0.90       153
                         macro avg       0.90      0.90      0.88       153
                      weighted avg       0.90      0.90      0.89       153

